In [ ]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import MetaTrader5 as mt5

from sesto.constants import CURRENCIES
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.indicators import SMA, EMA, RSI, ROC
from sesto.backtester import Backtester, Trade

In [ ]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

In [ ]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [ ]:
TIMEFRAME = mt5.TIMEFRAME_M15
BARS = 1500
MA_PERIODS = [7, 14, 21]
CSM_PERIOD = 21
CSM_TOP_THRESHOLD = 85
CSM_LOW_THRESHOLD = 15

In [ ]:
INITIAL_CAPITAL = 10000
TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001
TRADE_SL_MULTIPLIER = 0.03
RR_MULTIPLIER = 3

In [ ]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

In [ ]:
pairs_to_delete = [pair for pair, data in mtd.data.items() if data is None or len(data) == 0]
for pair in pairs_to_delete:
    del mtd.data[pair]

# Get the length of valid pair keys in mtd.data that have data
valid_pairs = sum(1 for pair, data in mtd.data.items() if data is not None and len(data) > 0)
print(f"Number of valid pairs with data: {valid_pairs}")

In [ ]:
for symbol, df in mtd.data.items():
    for period in MA_PERIODS:
        SMA(df, period)
        EMA(df, period)
        ROC(df, period)
        RSI(df, period)
    
    df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2
    # Smooth PSM
    df['psm'] = df['psm'].rolling(CSM_PERIOD).mean()


In [ ]:
csm_data = {}

for currency in CURRENCIES:
    # Initialize a list to hold DataFrames of PSM values for the current currency
    psm_dfs = []
    
    # Iterate over all pair data
    for pair, df in mtd.data.items():
        if df is None or df.empty:
            continue
        
        base_currency = pair[:3]
        quote_currency = pair[3:]
        
        # Determine if the current currency is the base or quote in the pair
        if currency == base_currency:
            # Use PSM as is for base currency
            psm_series = df[['time', 'psm']].copy()
        elif currency == quote_currency:
            # Invert PSM for quote currency
            psm_series = df[['time', 'psm']].copy()
            psm_series['psm'] = -psm_series['psm']
        else:
            # Current currency not involved in this pair
            continue
        
        # Rename the 'psm' column to the pair name
        psm_series.rename(columns={'psm': pair}, inplace=True)
        
        # Append to the list of PSM DataFrames
        psm_dfs.append(psm_series)
    
    if not psm_dfs:
        print(f"No pairs found for currency: {currency}")
        continue
    
    # Merge all PSM DataFrames on 'time' using outer join to preserve all timestamps
    csm_df = psm_dfs[0]
    for psm_df in psm_dfs[1:]:
        csm_df = pd.merge(csm_df, psm_df, on='time', how='outer')
    
    # Sort by time to maintain chronological order
    csm_df.sort_values('time', inplace=True)
    
    # Reset index after sorting
    csm_df.reset_index(drop=True, inplace=True)
    
    # Handle missing PSM values by excluding them from the mean calculation
    # (NaNs are automatically ignored by pandas' mean function with skipna=True)
    
    # Calculate the mean PSM value across all relevant pairs for each candle
    csm_df['csm'] = csm_df.drop('time', axis=1).mean(axis=1)

    min_csm = csm_df['csm'].min()
    max_csm = csm_df['csm'].max()
    csm_df['csm'] = 100 * (csm_df['csm'] - min_csm) / (max_csm - min_csm)
    
    # Store the CSM DataFrame in the dictionary
    csm_data[currency] = csm_df

In [ ]:
currencies_to_plot = ['GBP', 'JPY']  # You can modify this list as needed

# Create a new DataFrame to hold CSM data for all specified currencies
combined_csm_df = pd.DataFrame()

for currency in currencies_to_plot:
    if currency in csm_data:
        # Add the CSM data for this currency to the combined DataFrame
        combined_csm_df[currency] = csm_data[currency]['csm']
        
        # If we haven't set the 'time' column yet, use this currency's time data
        if 'time' not in combined_csm_df.columns:
            combined_csm_df['time'] = csm_data[currency]['time']

# Sort the combined DataFrame by time
combined_csm_df.sort_values('time', inplace=True)

# Plot the combined CSM data
plot_plotly(combined_csm_df, 'Combined CSM', currencies_to_plot)

In [ ]:

def get_csm(symbol: str, time: datetime, row: pd.Series):
    base_currency = symbol[:3]
    quote_currency = symbol[3:]

    base_csm_df = csm_data[base_currency]
    quote_csm_df = csm_data[quote_currency]

    base_csm_row_at_time = base_csm_df.loc[base_csm_df['time'] == row['time']].iloc[0]
    quote_csm_row_at_time = quote_csm_df.loc[quote_csm_df['time'] == row['time']].iloc[0]

    base_csm = base_csm_row_at_time['csm']
    quote_csm = quote_csm_row_at_time['csm']

    return base_csm, quote_csm

class MyStrategy(Backtester):
    def entry_condition(self, symbol: str, time: datetime, row: pd.Series, open_trades: List[Trade], all_trades: List[Trade]) -> bool:
        base_csm, quote_csm = get_csm(symbol, time, row)
        
        symbol_open_trades = [trade for trade in open_trades if trade.symbol == symbol]

        if len(symbol_open_trades) == 0:
            if base_csm > CSM_TOP_THRESHOLD and quote_csm < CSM_LOW_THRESHOLD:
                return {
                    'entry_price': row['close'],
                    'position_type': 'short',
                    'volume': 0.01,
                    'tp': row['close'] * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 + TRADE_SL_MULTIPLIER)
                }

            if base_csm < CSM_LOW_THRESHOLD and quote_csm > CSM_TOP_THRESHOLD:
                return {
                    'entry_price': row['close'],
                    'position_type': 'long',
                    'volume': 0.01,
                    'tp': row['close'] * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 - TRADE_SL_MULTIPLIER)
                }

        return None
    def exit_condition(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]) -> bool:
        # base_csm, quote_csm = get_csm(symbol, time= row['time'], row=row)

        # symbol_open_trades = [trade for trade in open_trades if trade.symbol == symbol]

        # if trade.position_type == 'short':
        #     if base_csm < CSM_LOW_THRESHOLD or quote_csm > CSM_TOP_THRESHOLD:
        #         return True
        # if trade.position_type == 'long':
        #     if base_csm > CSM_TOP_THRESHOLD or quote_csm < CSM_LOW_THRESHOLD:
        #         return True

        return False

    def trailing_stop(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]):
        # Implement your trailing stop logic here
        pass

In [ ]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=500.0  # Set your brokerage's leverage
)
backtest.run()

In [ ]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log

In [ ]:
performance_report = backtest.generate_report()
performance_report